In [ ]:
!pip install pandas requests beautifulsoup4 tqdm


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nptel_urls.csv to nptel_urls.csv


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [ ]:
filename = list(uploaded.keys())[0]  # Get the uploaded file name
df = pd.read_csv(filename)  # Read the CSV
urls = df.iloc[:, 0].dropna().tolist()  # First column → list of URLs

print(f"Total URLs found: {len(urls)}")
print(urls[:5])  # Preview first 5 URLs


Total URLs found: 924
['https://nptel.ac.in/courses/101106082', 'https://nptel.ac.in/courses/101101079', 'https://nptel.ac.in/courses/101101083', 'https://nptel.ac.in/courses/101104084', 'https://nptel.ac.in/courses/101104450']


In [ ]:
df = pd.read_csv(filename)
urls = df.iloc[:, 0].dropna().tolist()

data_list = []
error_list = []

print("Fetching details for each course...\n")

# ✅ Loop through each course URL
for url in tqdm(urls):
    try:
        # Extract numeric course ID
        course_id = url.strip().split("/")[-1]

        # Get course title from the course page
        r_page = requests.get(url, timeout=10)
        soup = BeautifulSoup(r_page.text, "html.parser")
        course_title_tag = soup.find("h1")
        course_title = course_title_tag.text.strip() if course_title_tag else "N/A"

        # Fetch stats from API
        api_url = f"https://tools.nptel.ac.in/npteldata/stats.php?id={course_id}"
        r_api = requests.get(api_url, timeout=10)
        r_api.raise_for_status()
        json_data = r_api.json()

        # ✅ Check if data exists
        if json_data.get("message") == "Success" and json_data.get("data"):
            base_data = json_data["data"][0]

            # Loop through each run (term)
            for run in base_data.get("run_wise_stats", []):
                data_list.append({
                    "Course URL": url,
                    "Course ID": course_id,
                    "Course Title": course_title,
                    "Enrolled": run.get("Enrolled"),
                    "Registered": run.get("Registered"),
                    "Gold": run.get("Gold"),
                    "Silver": run.get("Silver"),
                    "Elite": run.get("Elite"),
                    "Success": run.get("Success"),
                    "Max Mark": run.get("max_mark"),
                    "Min Mark": run.get("min_mark"),
                    "Average": run.get("average")
                })
        else:
            error_list.append({"Course URL": url, "Reason": "No data found"})

        time.sleep(0.3)  # Prevent overload

    except Exception as e:
        error_list.append({"Course URL": url, "Reason": str(e)})

# ✅ Save results to CSV
output_df = pd.DataFrame(data_list)
output_file = "nptel_selected_stats.csv"
output_df.to_csv(output_file, index=False)
print("\n✅ All data saved to:", output_file)

# ✅ Save errors (if any)
if error_list:
    error_df = pd.DataFrame(error_list)
    error_file = "nptel_errors.csv"
    error_df.to_csv(error_file, index=False)
    print("⚠ Errors logged in:", error_file)

# ✅ Download both files
files.download(output_file)
if error_list:
    files.download(error_file)

Fetching details for each course...



 98%|█████████▊| 907/924 [21:35<00:22,  1.30s/it]

In [ ]:
from google.colab import files
files.download("nptel_selected_stats.csv")
files.download("nptel_errors.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

git config --global user.email "fionadsouza1337@gmail.com"
  git config --global user.name "jadebluestar"

In [ ]:
!ls


sample_data


In [ ]:
output_file = "nptel_final_stats.json"
merged_df.to_csv(output_file, index=False)

files.download(output_file)
print("\n✅ Merged file is ready and downloaded.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


file downloaded!.


In [ ]:
import pandas as pd

# Load the merged CSV file
csv_file = "nptel_final_stats.csv"  # Replace with your actual CSV filename
df = pd.read_csv(csv_file)

# Convert to JSON format for the dashboard
output_file = "data.json"
df.to_json(output_file, orient="records", indent=2)

print(f"✅ JSON file '{output_file}' created successfully!")


✅ JSON file 'data.json' created successfully!


In [ ]:
files.download(output_file)
print("\nthe json file")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


the json file
